In [1]:
#
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
# join with animal df to get calving date
# dir = os.path.join(os.path.normpath(dirpath + os.sep + os.pardir),"udder_processing",  "udder_features")
animaldf = pd.read_csv("animal_info.csv", sep = ";")
animaldf.columns = [col.replace(" ", "_").lower().replace("-", "") for col in animaldf.columns]
fresh_dates = animaldf[["animal_number", "last_calving_date", "lactation_number"]]
fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')
fresh_dates = fresh_dates[~fresh_dates.fresh_date.isnull()]
fresh_dates = fresh_dates.drop("last_calving_date", axis = 1)
# filter records only on current lactation 
# get kickoffs current lactation per quarter

C:\Users\marie\AppData\Local\Temp\ipykernel_19596\1852963493.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')


In [3]:
# filter records only on current lactation
df = pd.read_csv(r"kick_offs\kick_offs_year.csv", sep = ";")
df.columns = [col.replace(" ", "_").lower().replace("-", "") for col in df.columns]

# [re.findall("U", val) for val in df.kickoff_lr]
df["ko_lr"] = [0 if "U" in val else 1 for val in df.kickoff_lr]
df["ko_lf"] = [0 if "U" in val else 1 for val in df.kickoff_lf]
df["ko_rf"] = [0 if "U" in val else 1 for val in df.kickoff_rf]
df["ko_rr"] = [0 if "U" in val else 1 for val in df.kickoff_rr]

df = df.drop(["kickoff_rf", "kickoff_lf", "kickoff_lr", "kickoff_rr"], axis = 1)

df["date"] = pd.to_datetime(df["date"], format = '%m/%d/%Y')
merged_df =  pd.merge(fresh_dates, df,  left_on="animal_number", right_on = "animal_number")

merged_df["dim"] = (merged_df["date"] - merged_df["fresh_date"]).dt.days
merged_df["total"] = merged_df["ko_lf"] + merged_df["ko_rf"] + merged_df["ko_rr"]+ merged_df["ko_lr"]
merged_df = merged_df[merged_df.dim >= 0]

## total kick offs (all visits) current lactation

In [4]:
ko_df = merged_df.groupby(["animal_number", "lactation_number"])[["ko_lr","ko_lf", "ko_rf","ko_rr"]].agg("sum").reset_index()
dim_df =  merged_df.groupby("animal_number")[["dim"]].agg(["max", "count"]).droplevel(axis=1, level=0).reset_index()
ko_df2 = pd.merge(ko_df, dim_df,  left_on="animal_number", right_on = "animal_number")
ko_df2["udder_ko"] = ko_df2.ko_rr + ko_df2.ko_rf +ko_df2.ko_lr + ko_df2.ko_lf

ko_df2.to_csv("visit_kick_offs_current_lactaion.csv", index = False)
visit = ko_df2

## days qith kick offs current lactation 

In [7]:
ko_df = merged_df.groupby(["animal_number", "lactation_number", "date"])[["ko_lr","ko_lf", "ko_rf","ko_rr"]].agg("sum").reset_index()
ko_df2 = ko_df.copy()
ko_df2["dko_lr"] = (ko_df["ko_lr"] > 0).astype(int)
ko_df2["dko_lf"] = (ko_df["ko_lf"] > 0).astype(int)
ko_df2["dko_rf"] = (ko_df["ko_rf"] > 0).astype(int)
ko_df2["dko_rr"] = (ko_df["ko_rr"] > 0).astype(int)

ko_df2["udder_dko"] = ko_df2.ko_rr + ko_df2.ko_rf +ko_df2.ko_lr + ko_df2.ko_lf
ko_df2["total_dko"] = (ko_df2["udder_dko"] > 0).astype(int)

ko_df3 = ko_df2.groupby(["animal_number", "lactation_number"])[["ko_lr","ko_lf", "ko_rf","ko_rr", "udder_dko", "total_dko"]].agg("sum").reset_index()
ko_df4 = pd.merge(ko_df3, dim_df,  left_on="animal_number", right_on = "animal_number")

ko_df4["ko_days_pc"] = (ko_df4["total_dko"]/ko_df4["max"])*100

ko_df4.to_csv("day_kick_offs_current_lactaion.csv", index = False)

## last 7 days (2023-11-9 to 2023-11-16)

In [19]:
merged_df7 = merged_df[(merged_df.date >= '2023-11-9') & (merged_df.date < '2023-11-17')]
ko_df = merged_df7.groupby(["animal_number", "lactation_number", "date"])[["ko_lr","ko_lf", "ko_rf","ko_rr"]].agg("sum").reset_index()
ko_df2 = ko_df.copy()
ko_df2["dko_lr"] = (ko_df["ko_lr"] > 0).astype(int)
ko_df2["dko_lf"] = (ko_df["ko_lf"] > 0).astype(int)
ko_df2["dko_rf"] = (ko_df["ko_rf"] > 0).astype(int)
ko_df2["dko_rr"] = (ko_df["ko_rr"] > 0).astype(int)

ko_df2["udder_dko"] = ko_df2.ko_rr + ko_df2.ko_rf +ko_df2.ko_lr + ko_df2.ko_lf
ko_df2["total_dko"] = (ko_df2["udder_dko"] > 0).astype(int)

ko_df3 = ko_df2.groupby(["animal_number", "lactation_number"])[["ko_lr","ko_lf", "ko_rf","ko_rr", "udder_dko", "total_dko"]].agg("sum").reset_index()
ko_df4 = pd.merge(ko_df3, dim_df,  left_on="animal_number", right_on = "animal_number")

ko_df4["ko_days_pc"] = (ko_df4["total_dko"]/7)*100

ko_df4.to_csv("day_kick_offs_last_7d.csv", index = False)